In [16]:
from langchain_community.document_loaders import TextLoader # Take raw text of descriptions and convert into format langchain can work with

from langchain_text_splitters import CharacterTextSplitter # Split whole document containing descriptions into meaningful chunks
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

In [17]:
from dotenv import load_dotenv
load_dotenv()

True

In [18]:
import pandas as pd
movies = pd.read_csv('movies-extended.csv')

In [19]:
movies

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Title-And-Plot,Genre-And-Plot,New-Genre
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",Kansas Saloon Smashers : A bartender is workin...,"A bartender is working at a saloon, serving dr...",NaN
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...","Love by the Light of the Moon : The moon, pain...","The moon, painted with a smiling face hangs ov...",NaN
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...","The Martyred Presidents : The film, just over ...","The film, just over a minute long, is composed...",NaN
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,"Terrible Teddy, the Grizzly King : Lasting jus...",Lasting just 61 seconds and consisting of two ...,NaN
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,Jack and the Beanstalk : The earliest known ad...,The earliest known adaptation of the classic f...,NaN
...,...,...,...,...,...,...,...,...,...,...,...
22341,2017,"Hochelaga, Land of Souls (Hochelaga terre des ...",Canadian,François Girard,"Raoul Max Trujillo, Tanaya Beatty, David La Haye",historical drama,"https://en.wikipedia.org/wiki/Hochelaga,_Land_...","One night on the campus of McGill University, ...","Hochelaga, Land of Souls (Hochelaga terre des ...",historical drama : One night on the campus of ...,historical
22342,2017,Indian Horse,Canadian,Stephen Campanelli,"Forrest Goodluck, Michiel Huisman, Michael Mur...",drama,https://en.wikipedia.org/wiki/Indian_Horse_(film),"The Indian Horse family, including six-year-ol...","Indian Horse : The Indian Horse family, includ...","drama : The Indian Horse family, including six...",drama
22343,2017,The Little Girl Who Was Too Fond of Matches (L...,Canadian,Simon Lavoie,NaN,unknown,https://en.wikipedia.org/wiki/The_Little_Girl_...,"In rural 1930s Quebec, Alice lives in house wi...",The Little Girl Who Was Too Fond of Matches (L...,"In rural 1930s Quebec, Alice lives in house wi...",NaN
22344,2017,Meditation Park,Canadian,Mina Shum,"Sandra Oh, Liane Balaban, Don McKellar",drama,https://en.wikipedia.org/wiki/Meditation_Park,"Opened by Mandarin theme song, Meditation Park...",Meditation Park : Opened by Mandarin theme son...,"drama : Opened by Mandarin theme song, Meditat...",drama


In [20]:
movies['movie_id'] = movies.index

In [21]:
movies

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Title-And-Plot,Genre-And-Plot,New-Genre,movie_id
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",Kansas Saloon Smashers : A bartender is workin...,"A bartender is working at a saloon, serving dr...",NaN,0
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...","Love by the Light of the Moon : The moon, pain...","The moon, painted with a smiling face hangs ov...",NaN,1
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...","The Martyred Presidents : The film, just over ...","The film, just over a minute long, is composed...",NaN,2
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,"Terrible Teddy, the Grizzly King : Lasting jus...",Lasting just 61 seconds and consisting of two ...,NaN,3
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,Jack and the Beanstalk : The earliest known ad...,The earliest known adaptation of the classic f...,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...
22341,2017,"Hochelaga, Land of Souls (Hochelaga terre des ...",Canadian,François Girard,"Raoul Max Trujillo, Tanaya Beatty, David La Haye",historical drama,"https://en.wikipedia.org/wiki/Hochelaga,_Land_...","One night on the campus of McGill University, ...","Hochelaga, Land of Souls (Hochelaga terre des ...",historical drama : One night on the campus of ...,historical,22341
22342,2017,Indian Horse,Canadian,Stephen Campanelli,"Forrest Goodluck, Michiel Huisman, Michael Mur...",drama,https://en.wikipedia.org/wiki/Indian_Horse_(film),"The Indian Horse family, including six-year-ol...","Indian Horse : The Indian Horse family, includ...","drama : The Indian Horse family, including six...",drama,22342
22343,2017,The Little Girl Who Was Too Fond of Matches (L...,Canadian,Simon Lavoie,NaN,unknown,https://en.wikipedia.org/wiki/The_Little_Girl_...,"In rural 1930s Quebec, Alice lives in house wi...",The Little Girl Who Was Too Fond of Matches (L...,"In rural 1930s Quebec, Alice lives in house wi...",NaN,22343
22344,2017,Meditation Park,Canadian,Mina Shum,"Sandra Oh, Liane Balaban, Don McKellar",drama,https://en.wikipedia.org/wiki/Meditation_Park,"Opened by Mandarin theme song, Meditation Park...",Meditation Park : Opened by Mandarin theme son...,"drama : Opened by Mandarin theme song, Meditat...",drama,22344


In [22]:
movies['id-title-plot'] = movies['movie_id'].astype(str) + ' ' + movies['Title'] + ' - ' + movies['Plot']

In [23]:
for col in movies.select_dtypes(include=["object"]).columns:
    movies[col] = movies[col].str.replace("\n", " ", regex=False)
    movies[col] = movies[col].str.replace("\r", " ", regex=False)

In [46]:
movies['id-title-plot'] = movies['id-title-plot'].str.slice(0, 1000)

In [47]:
movies

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Title-And-Plot,Genre-And-Plot,New-Genre,movie_id,id-title-plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",Kansas Saloon Smashers : A bartender is workin...,"A bartender is working at a saloon, serving dr...",NaN,0,0 Kansas Saloon Smashers - A bartender is work...
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...","Love by the Light of the Moon : The moon, pain...","The moon, painted with a smiling face hangs ov...",NaN,1,"1 Love by the Light of the Moon - The moon, pa..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...","The Martyred Presidents : The film, just over ...","The film, just over a minute long, is composed...",NaN,2,"2 The Martyred Presidents - The film, just ove..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,"Terrible Teddy, the Grizzly King : Lasting jus...",Lasting just 61 seconds and consisting of two ...,NaN,3,"3 Terrible Teddy, the Grizzly King - Lasting j..."
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,Jack and the Beanstalk : The earliest known ad...,The earliest known adaptation of the classic f...,NaN,4,4 Jack and the Beanstalk - The earliest known ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22341,2017,"Hochelaga, Land of Souls (Hochelaga terre des ...",Canadian,François Girard,"Raoul Max Trujillo, Tanaya Beatty, David La Haye",historical drama,"https://en.wikipedia.org/wiki/Hochelaga,_Land_...","One night on the campus of McGill University, ...","Hochelaga, Land of Souls (Hochelaga terre des ...",historical drama : One night on the campus of ...,historical,22341,"22341 Hochelaga, Land of Souls (Hochelaga terr..."
22342,2017,Indian Horse,Canadian,Stephen Campanelli,"Forrest Goodluck, Michiel Huisman, Michael Mur...",drama,https://en.wikipedia.org/wiki/Indian_Horse_(film),"The Indian Horse family, including six-year-ol...","Indian Horse : The Indian Horse family, includ...","drama : The Indian Horse family, including six...",drama,22342,"22342 Indian Horse - The Indian Horse family, ..."
22343,2017,The Little Girl Who Was Too Fond of Matches (L...,Canadian,Simon Lavoie,NaN,unknown,https://en.wikipedia.org/wiki/The_Little_Girl_...,"In rural 1930s Quebec, Alice lives in house wi...",The Little Girl Who Was Too Fond of Matches (L...,"In rural 1930s Quebec, Alice lives in house wi...",NaN,22343,22343 The Little Girl Who Was Too Fond of Matc...
22344,2017,Meditation Park,Canadian,Mina Shum,"Sandra Oh, Liane Balaban, Don McKellar",drama,https://en.wikipedia.org/wiki/Meditation_Park,"Opened by Mandarin theme song, Meditation Park...",Meditation Park : Opened by Mandarin theme son...,"drama : Opened by Mandarin theme song, Meditat...",drama,22344,22344 Meditation Park - Opened by Mandarin the...


In [67]:
movies.to_csv('movies_semantic.csv', index=False)

In [48]:
movies['id-title-plot'].to_csv('movies_semantic.txt', index=False, header=False)

In [49]:
from langchain.docstore.document import Document

documents = []
with open('movies_semantic.txt', 'r', encoding='utf-8') as file:
    for line in file:
        line = line.strip()
        if line:
            documents.append(Document(page_content=line))

In [50]:
documents[12307]

Document(metadata={}, page_content='"12307 The Shawshank Redemption - In 1947 Portland, Maine, banker Andy Dufresne is convicted of murdering his wife and her lover, and sentenced to two consecutive life sentences at the Shawshank State Penitentiary. He is befriended by contraband smuggler, Ellis ""Red"" Redding, an inmate serving a life sentence. Red procures a rock hammer, and later a large poster of Rita Hayworth for Andy. Working in the prison laundry, Andy is regularly assaulted and raped by ""the Sisters"" and their leader, Bogs.  In 1949, Andy overhears the captain of the guards, Byron Hadley, complaining about being taxed on an inheritance and offers to help him shelter the money legally. After an assault by the Sisters nearly kills Andy, Hadley beats Bogs severely. Left crippled, Bogs is transferred to another prison, and Andy is not attacked again. Warden Samuel Norton meets Andy and reassigns him to the prison library to assist elderly inmate Brooks Hatlen. Andy\'s new job i

In [51]:
db_movies = Chroma.from_documents(documents, embedding=OpenAIEmbeddings())

In [65]:
query = 'Jim Carrey'
docs = db_movies.similarity_search(query, k=5)

In [61]:
docs

[Document(id='053a9c3c-1bb1-4687-8d11-3bfd0d8529b6', metadata={}, page_content='"8993 The Way We Were - Told partly in flashback, it is the story of Katie Morosky (Barbra Streisand) and Hubbell Gardiner (Robert Redford). Their differences are immense: she is a stridently vocal Marxist Jew with strong anti-war opinions, and he is a carefree WASP with no particular political bent. While attending the same college, she is drawn to him because of his boyish good looks and his natural writing skill, which she finds captivating, although he does not work very hard at it. He is intrigued by her conviction and her determination to persuade others to take up social causes. Their attraction is evident, but neither of them acts upon it, and they lose touch after graduation.  The two meet again towards the end of World War II while Katie is working at a radio station, and Hubbell, having served as a naval officer in the South Pacific, is trying to return to civilian life. They fall in love despite

In [58]:
def retrieve_semantic_recommendations(query, top_k=10):
    recs = db_movies.similarity_search(query, k=top_k)

    movies_list = []

    for i in range(0, len(recs)):
        raw_text = recs[i].page_content.strip('"').strip()
        first_token, rest = raw_text.split(' ', 1)
        id = int(first_token)
        movies_list.append(id)

    return movies[movies["movie_id"].isin(movies_list)]

In [66]:
retrieve_semantic_recommendations(query, top_k=5)

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Title-And-Plot,Genre-And-Plot,New-Genre,movie_id,id-title-plot
2112,1936,Picadilly Jim,American,Robert Z. Leonard,"Robert Montgomery, Frank Morgan, Billie Burke",romantic comedy,https://en.wikipedia.org/wiki/Piccadilly_Jim_(...,"In London, American caricaturist Jim Crocker (...","Picadilly Jim : In London, American caricaturi...","romantic comedy : In London, American caricatu...",NaN,2112,"2112 Picadilly Jim - In London, American caric..."
12179,1994,Dumb and Dumber,American,"Peter Farrelly, Bobby Farrelly","Jim Carrey, Jeff Daniels, Lauren Holly",comedy,https://en.wikipedia.org/wiki/Dumb_and_Dumber,Lloyd Christmas (Jim Carrey) and Harry Dunne (...,Dumb and Dumber : Lloyd Christmas (Jim Carrey)...,comedy : Lloyd Christmas (Jim Carrey) and Harr...,comedy,12179,12179 Dumb and Dumber - Lloyd Christmas (Jim C...
14220,2003,Bruce Almighty,American,Tom Shadyac,"Jim Carrey, Morgan Freeman, Jennifer Aniston, ...","comedy, fantasy",https://en.wikipedia.org/wiki/Bruce_Almighty,Bruce Nolan (Jim Carrey) is a television field...,Bruce Almighty : Bruce Nolan (Jim Carrey) is a...,"comedy, fantasy : Bruce Nolan (Jim Carrey) is ...",NaN,14220,14220 Bruce Almighty - Bruce Nolan (Jim Carrey...
21683,1981,Rubberface,Canadian,"Glen Salzman, Rebecca Yates",Jim Carrey,comedy,https://en.wikipedia.org/wiki/Rubberface,A funny schoolgirl (Adah Glassbourg) becomes f...,Rubberface : A funny schoolgirl (Adah Glassbou...,comedy : A funny schoolgirl (Adah Glassbourg) ...,comedy,21683,21683 Rubberface - A funny schoolgirl (Adah Gl...
21705,1983,All in Good Taste,Canadian,Anthony Kramreither,"Jonathan Welsh, Jim Carrey",comedy,https://en.wikipedia.org/wiki/All_in_Good_Taste,The young filmmaker Timothy (Jonathan Welsh) h...,All in Good Taste : The young filmmaker Timoth...,comedy : The young filmmaker Timothy (Jonathan...,comedy,21705,21705 All in Good Taste - The young filmmaker ...
